In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import sqlite3
from datetime import datetime
#from matplotlib.widgets import Dropdown

In [8]:
# Connect to the SQLite database
conn = sqlite3.connect('data/sp500_data.db')

In [9]:
# Ask the user for input
symbol = input("Enter the stock symbol: ")
start_date_str = input("Enter the start date (YYYY-MM-DD): ")
end_date_str = input("Enter the end date (YYYY-MM-DD): ")

In [10]:
# Convert the input strings to datetime objects
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')


In [11]:
# Construct the SQL query
sql_query = f"SELECT * FROM stocks WHERE Symbol = ? AND Date >= ? AND Date <= ?"

In [12]:
# Execute the query with user input
cursor = conn.cursor()
cursor.execute(sql_query, (symbol, start_date_str, end_date_str))
# Fetch all the rows
rows = cursor.fetchall()


In [15]:
# Create a DataFrame from the query results
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Shortname
0,2022-01-03 00:00:00.000000,MMM,176.038330,177.740005,179.089996,175.839996,178.320007,1930700.0,3M Company
1,2022-01-04 00:00:00.000000,MMM,178.504471,180.229996,181.259995,178.029999,178.479996,2522200.0,3M Company
2,2022-01-05 00:00:00.000000,MMM,177.771576,179.490005,181.779999,177.000000,177.130005,2952400.0,3M Company
3,2022-01-06 00:00:00.000000,MMM,176.295822,178.000000,181.279999,177.539993,180.880005,2505400.0,3M Company
4,2022-01-07 00:00:00.000000,MMM,178.227158,179.949997,180.490005,177.220001,178.130005,2800200.0,3M Company
5,2022-01-10 00:00:00.000000,MMM,175.711487,177.410004,179.830002,176.779999,179.449997,2318000.0,3M Company
6,2022-01-11 00:00:00.000000,MMM,177.266449,178.979996,179.550003,176.419998,178.000000,2264200.0,3M Company
7,2022-01-12 00:00:00.000000,MMM,177.702225,179.419998,180.009995,178.589996,179.000000,1728500.0,3M Company
8,2022-01-13 00:00:00.000000,MMM,179.286926,181.020004,181.339996,179.029999,179.880005,2094700.0,3M Company
9,2022-01-14 00:00:00.000000,MMM,177.028748,178.740005,180.860001,177.669998,180.860001,2286100.0,3M Company


In [16]:
# Add a percent change column to the DataFrame
df['Percent_Change'] = df['Adj Close'].pct_change() * 100
# Replace all NaN values in the Percent_Change column with 0
df['Percent_Change'].fillna(0, inplace=True)

In [17]:
# Print the DataFrame
print(df)

                          Date Symbol   Adj Close       Close        High  \
0   2022-01-03 00:00:00.000000    MMM  176.038330  177.740005  179.089996   
1   2022-01-04 00:00:00.000000    MMM  178.504471  180.229996  181.259995   
2   2022-01-05 00:00:00.000000    MMM  177.771576  179.490005  181.779999   
3   2022-01-06 00:00:00.000000    MMM  176.295822  178.000000  181.279999   
4   2022-01-07 00:00:00.000000    MMM  178.227158  179.949997  180.490005   
5   2022-01-10 00:00:00.000000    MMM  175.711487  177.410004  179.830002   
6   2022-01-11 00:00:00.000000    MMM  177.266449  178.979996  179.550003   
7   2022-01-12 00:00:00.000000    MMM  177.702225  179.419998  180.009995   
8   2022-01-13 00:00:00.000000    MMM  179.286926  181.020004  181.339996   
9   2022-01-14 00:00:00.000000    MMM  177.028748  178.740005  180.860001   
10  2022-01-18 00:00:00.000000    MMM  176.573151  178.279999  178.550003   
11  2022-01-19 00:00:00.000000    MMM  176.771225  178.479996  180.869995   

In [ ]:
# Plot the S&P 500 and Apple stock prices
plt.figure(figsize=(10, 5))
plt.plot(sp500_close, label='S&P 500')
plt.plot(apple_close, label='Apple (AAPL)')
plt.legend()
plt.title('S&P 500 vs. Apple Stock Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# Animate the plot
ani = FuncAnimation(fig, update, frames=len(sp500_df), blit=True, repeat=False)

plt.title('S&P 500 vs. Apple Stock Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

In [ ]:
# Importing module 
import mysql.connector
 
# Creating connection object
mydb = mysql.connector.connect(
    host = "localhost",
    user = "yourusername",
    password = "your_password"
)
 
# Printing the connection object 
print(mydb)